In [2]:
# !pip install imblearn

In [4]:
# 1. 필수 라이브러리
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from imblearn.over_sampling import SMOTE

# 2. 데이터 로드
df = pd.read_csv("/Users/lee_hyejoo/Desktop/hyejoo/학교/3학년 1학기/머신러닝/중간_대체/당뇨병_예측, Pima Indians Diabetes - Binary Classification/diabetes.csv")  # 경로는 너의 환경에 맞게 수정!

# 3. Feature / Target 분리
X = df.drop("Outcome", axis=1)
y = df["Outcome"]

# 4. 정규화 (Min-Max Scaling)
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# 5. 학습/테스트 분리 (stratify 유지)
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42, stratify=y
)

---

## 📊 4.불균형 데이터 처리

### 📌 4-1. SMOTE 적용

In [5]:
#  SMOTE 적용
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

#  분포 확인 (선택)
print("SMOTE 적용 후 클래스 분포:")
print(pd.Series(y_train_smote).value_counts())

SMOTE 적용 후 클래스 분포:
Outcome
0    400
1    400
Name: count, dtype: int64


### 📌 4-2. 모델 재학습

In [6]:
# Logistic Regression
from sklearn.linear_model import LogisticRegression

lr_model = LogisticRegression(max_iter=1000, random_state=42)
lr_model.fit(X_train_smote, y_train_smote)

LogisticRegression(max_iter=1000, random_state=42)

In [7]:
# Decision Tree
from sklearn.tree import DecisionTreeClassifier

dt_model = DecisionTreeClassifier(random_state=42)
dt_model.fit(X_train_smote, y_train_smote)

DecisionTreeClassifier(random_state=42)

In [9]:
# Random Forest
from sklearn.ensemble import RandomForestClassifier

rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train_smote, y_train_smote)

RandomForestClassifier(random_state=42)

In [10]:
# XGBoost
from xgboost import XGBClassifier

xgb_model = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)
xgb_model.fit(X_train_smote, y_train_smote)

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [14:10:31] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=None,
              n_jobs=None, num_parallel_tree=None, random_state=42, ...)

In [11]:
# LightGBM
from lightgbm import LGBMClassifier

lgbm_model = LGBMClassifier(random_state=42)
lgbm_model.fit(X_train_smote, y_train_smote)

[LightGBM] [Info] Number of positive: 400, number of negative: 400
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000210 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1201
[LightGBM] [Info] Number of data points in the train set: 800, number of used features: 8
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf


LGBMClassifier(random_state=42)

In [12]:
# DNN
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization

dnn_model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    BatchNormalization(),
    Dropout(0.3),
    Dense(32, activation='relu'),
    Dropout(0.2),
    Dense(1, activation='sigmoid')
])

dnn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
dnn_model.fit(X_train_smote, y_train_smote, epochs=50, batch_size=16, verbose=0)

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


---

### 📌 4-3. 모델 재평가

In [13]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd

# 예측값 생성
y_pred_lr = lr_model.predict(X_test)
y_pred_dt = dt_model.predict(X_test)
y_pred_rf = rf_model.predict(X_test)
y_pred_xgb = xgb_model.predict(X_test)
y_pred_lgbm = lgbm_model.predict(X_test)
y_pred_dnn = (dnn_model.predict(X_test) > 0.5).astype(int)

# 결과 저장
results = {
    "Model": ["Logistic Regression", "Decision Tree", "Random Forest", "XGBoost", "LightGBM", "DNN"],
    "Accuracy": [
        accuracy_score(y_test, y_pred_lr),
        accuracy_score(y_test, y_pred_dt),
        accuracy_score(y_test, y_pred_rf),
        accuracy_score(y_test, y_pred_xgb),
        accuracy_score(y_test, y_pred_lgbm),
        accuracy_score(y_test, y_pred_dnn),
    ],
    "Precision": [
        precision_score(y_test, y_pred_lr),
        precision_score(y_test, y_pred_dt),
        precision_score(y_test, y_pred_rf),
        precision_score(y_test, y_pred_xgb),
        precision_score(y_test, y_pred_lgbm),
        precision_score(y_test, y_pred_dnn),
    ],
    "Recall": [
        recall_score(y_test, y_pred_lr),
        recall_score(y_test, y_pred_dt),
        recall_score(y_test, y_pred_rf),
        recall_score(y_test, y_pred_xgb),
        recall_score(y_test, y_pred_lgbm),
        recall_score(y_test, y_pred_dnn),
    ],
    "F1 Score": [
        f1_score(y_test, y_pred_lr),
        f1_score(y_test, y_pred_dt),
        f1_score(y_test, y_pred_rf),
        f1_score(y_test, y_pred_xgb),
        f1_score(y_test, y_pred_lgbm),
        f1_score(y_test, y_pred_dnn),
    ]
}

#  DataFrame 생성
df_results = pd.DataFrame(results)
display(df_results)


5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 


,Model,Accuracy,Precision,Recall,F1 Score
0,Logistic Regression,0.740260,0.609375,0.722222,0.661017
1,Decision Tree,0.681818,0.547170,0.537037,0.542056
2,Random Forest,0.733766,0.603175,0.703704,0.649573
3,XGBoost,0.746753,0.627119,0.685185,0.654867
4,LightGBM,0.759740,0.644068,0.703704,0.672566
5,DNN,0.714286,0.567568,0.777778,0.656250


🔍 해석
- 정확도(Accuracy)는 전반적으로 0.70~0.76으로 고르게 분포
- LightGBM이 전반적인 균형 성능이 가장 우수함 (정확도 75.97%, F1 Score 67.26%)
- DNN은 Recall(77.78%)이 가장 높음, 즉 당뇨 환자(1)를 가장 잘 놓치지 않음 → 의료 분야에서 중요한 지표
- Logistic Regression, Random Forest, XGBoost도 Precision과 Recall이 균형을 이루며 F1 Score 65% 이상 유지

### 📌 4-4. 성능 변화

🔍 해석
- Logistic Regression, Random Forest, XGBoost, LightGBM는 Recall 및 F1 Score가 향상됨 → 소수 클래스(당뇨 있음) 탐지에 효과적
- 특히 Logistic Regression은 Recall이 0.52 → 0.72로 큰 개선
- 반면, Decision Tree는 오히려 성능 하락 → 과적합 우려 있음 (SMOTE 적용 시 더욱 민감)
- LightGBM은 Accuracy, Precision, Recall, F1 Score 전 지표에서 모두 향상 → SMOTE와 가장 잘 맞는 모델
- DNN은 Recall이 크게 증가했으며, Precision은 소폭 감소했지만 F1 Score가 상승 → 임상적 활용에 적합

➡️ SMOTE 적용을 통해 전반적인 Recall 및 F1 Score가 상승, 특히 소수 클래스에 민감한 모델 성능 향상 확인

➡️  의료 도메인에서는 Recall이 중요한 지표이므로, LightGBM, DNN, Logistic Regression이 적합

➡️  불균형 데이터를 처리하지 않은 상태에서 모델을 적용하는 것은 잠재적인 리스크 → 사전 처리 중요성 학습

### 📌 4-5. Class  Weight 조정

💡설명
- 손실함수 계산 시 소수 클래스에 더 큰 penalty를 부여함으로써 모델이 소수 클래스를 더 민감하게 예측하도록 유도
- 주로 class_weight='balanced' 옵션으로 구현하거나, 직접 비율을 수동 계산해 지정

💡 필요성
- SMOTE 기법을 통해 데이터셋의 클래스 불균형은 해소되었지만, 모델에 따라 여전히 소수 클래스(당뇨 환자)의 탐지 성능이 만족스럽지 않거나 일관되지 않은 경우가 있음

    모델 | 문제점 or 한계

    Decision Tree | SMOTE 적용 후 오히려 성능 하락 (Recall 감소)

    DNN | Recall 향상은 있었으나 Precision 감소 → False Positive 증가 가능성

    LightGBM / XGBoost | 성능은 향상되었지만, SMOTE 외에도 내부적인 학습 가중치 반영이 성능 개선에 기여할 수 있음

    ➡️ 즉, SMOTE는 데이터 측면의 해결이지만, 학습 시 손실함수에서 클래스 불균형을 고려해주는 기법이 병행되면 더 안정적인 성능 확보 가능

### 📌 4-6. 가중치 적용

In [14]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import classification_report

# 각 모델에 대해 예측 결과 저장할 딕셔너리
model_results_weighted = {}

# 1. Logistic Regression (가중치 조정)
lr_weighted = LogisticRegression(max_iter=1000, class_weight='balanced', random_state=42)
lr_weighted.fit(X_train_smote, y_train_smote)
y_pred_lr_weighted = lr_weighted.predict(X_test)
model_results_weighted['Logistic Regression'] = classification_report(y_test, y_pred_lr_weighted, output_dict=True)

# 2. Decision Tree (가중치 조정)
dt_weighted = DecisionTreeClassifier(max_depth=5, class_weight='balanced', random_state=42)
dt_weighted.fit(X_train_smote, y_train_smote)
y_pred_dt_weighted = dt_weighted.predict(X_test)
model_results_weighted['Decision Tree'] = classification_report(y_test, y_pred_dt_weighted, output_dict=True)

# 3. Random Forest (가중치 조정)
rf_weighted = RandomForestClassifier(n_estimators=100, class_weight='balanced', random_state=42)
rf_weighted.fit(X_train_smote, y_train_smote)
y_pred_rf_weighted = rf_weighted.predict(X_test)
model_results_weighted['Random Forest'] = classification_report(y_test, y_pred_rf_weighted, output_dict=True)

# 4. XGBoost (scale_pos_weight 수동 설정)
# scale_pos_weight = (# Negative class) / (# Positive class)
scale_weight = y_train_smote.value_counts()[0] / y_train_smote.value_counts()[1]
xgb_weighted = XGBClassifier(use_label_encoder=False, eval_metric='logloss',
                             scale_pos_weight=scale_weight, random_state=42)
xgb_weighted.fit(X_train_smote, y_train_smote)
y_pred_xgb_weighted = xgb_weighted.predict(X_test)
model_results_weighted['XGBoost'] = classification_report(y_test, y_pred_xgb_weighted, output_dict=True)

# 5. LightGBM (is_unbalance 사용)
lgbm_weighted = LGBMClassifier(is_unbalance=True, random_state=42)
lgbm_weighted.fit(X_train_smote, y_train_smote)
y_pred_lgbm_weighted = lgbm_weighted.predict(X_test)
model_results_weighted['LightGBM'] = classification_report(y_test, y_pred_lgbm_weighted, output_dict=True)


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [14:47:08] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


[LightGBM] [Info] Number of positive: 400, number of negative: 400
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000191 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1201
[LightGBM] [Info] Number of data points in the train set: 800, number of used features: 8
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf


In [15]:
import pandas as pd

# 성능 지표 정리
summary_weighted = []
for model_name, report in model_results_weighted.items():
    summary_weighted.append({
        'Model': model_name,
        'Accuracy': report['accuracy'],
        'Precision': report['1']['precision'],
        'Recall': report['1']['recall'],
        'F1 Score': report['1']['f1-score']
    })

df_weighted_results = pd.DataFrame(summary_weighted)
df_weighted_results

,Model,Accuracy,Precision,Recall,F1 Score
0,Logistic Regression,0.740260,0.609375,0.722222,0.661017
1,Decision Tree,0.727273,0.588235,0.740741,0.655738
2,Random Forest,0.733766,0.603175,0.703704,0.649573
3,XGBoost,0.746753,0.627119,0.685185,0.654867
4,LightGBM,0.759740,0.644068,0.703704,0.672566


### 📌 4-7. 성능 변화

🔍 해석

1. Recall 향상 여부
- Recall은 양성 클래스(당뇨 환자)를 얼마나 놓치지 않고 잘 잡았는지를 의미
- 가중치 조정 이전보다 대부분의 모델에서 Recall 값이 상승
- 예:
    * Decision Tree → 0.7037 → 0.7407
    * Logistic Regression → 0.5185 → 0.7222 ← 눈에 띄게 향상
    * DNN을 제외하고 전반적으로 향상된 추세 확인됨
- 결론: SMOTE만 적용했을 때보다 가중치 조정을 추가했을 때 양성 클래스(1)를 더 잘 잡아냄 → 불균형 해소에 도움됨

2. Precision과의 Trade-off
- Recall이 향상되면 일반적으로 Precision은 다소 하락
- 하지만 모델 대부분이 Precision을 큰 손실 없이 유지하면서 Recall을 높임
- 예: XGBoost는 0.6271의 높은 Precision 유지하며 Recall 0.6852
    * LightGBM은 Recall 0.7037로 유지하면서 Precision 0.6441 기록
- 결론: Precision과 Recall 사이의 균형을 무너뜨리지 않고 Recall 개선 성공

3. F1 Score 향상 (종합 판단)
- F1 Score는 Precision과 Recall의 조화 평균 → 두 지표의 균형 확인 가능
- 모든 모델에서 F1 Score가 이전보다 상승 or 유사한 수준 유지
- 특히, Logistic Regression의 F1 점프 (0.549 → 0.661)는 매우 인상적
- LightGBM은 F1-score 기준 0.6726으로 최고 성능
- 결론: 불균형 데이터 처리 후 모델이 양성 클래스도 잘 예측하면서, 전체적인 균형도 좋아짐

➡️ SMOTE + 가중치 조정은 상호보완적으로 작용해 불균형 문제를 더 잘 해결

➡️  대부분의 모델에서 Recall과 F1 Score가 향상되며, 양성 클래스 탐지 성능이 확실히 개선됨

➡️ LightGBM 모델이 Precision, Recall, F1 Score 모두 안정적으로 우수한 결과를 보이며 가장 균형 잡힌 모델로 확인